# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [182]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [183]:
housing = pd.read_csv('cal_housing_clean.csv')

In [184]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [185]:
housing.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [186]:
x_data = housing.drop('medianHouseValue', axis=1)

In [187]:
y_val = housing['medianHouseValue']

In [188]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3, random_state=101)

In [189]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(14448, 6)
(6192, 6)
(14448,)
(6192,)


### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [190]:
from sklearn.preprocessing import MinMaxScaler

In [191]:
scaler = MinMaxScaler()

In [192]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [193]:
X_train = pd.DataFrame(data=scaler.transform(X_train),
                       columns=X_train.columns,
                       index=X_train.index)
X_test = pd.DataFrame(data=scaler.transform(X_test),
                      columns=X_test.columns,
                      index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [194]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [195]:
import tensorflow as tf

In [196]:
housingMedianAge = tf.feature_column.numeric_column('housingMedianAge')
totalRooms = tf.feature_column.numeric_column('totalRooms')
totalBedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medianIncome = tf.feature_column.numeric_column('medianIncome')

In [197]:
feat_cols = [housingMedianAge, totalRooms, totalBedrooms, population, households, medianIncome]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [198]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [239]:
# model = tf.estimator.DNNRegressor(hidden_units=[6, 6, 6], feature_columns=feat_cols)
model = tf.estimator.LinearRegressor(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xt/qvsnsqc941q950ghsjhyjh3r0000gn/T/tmpq9qdberp', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_session_config': None, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_tf_random_seed': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [240]:
model.train(input_fn=input_func, steps=100000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/xt/qvsnsqc941q950ghsjhyjh3r0000gn/T/tmpq9qdberp/model.ckpt.
INFO:tensorflow:loss = 7.33496e+12, step = 1
INFO:tensorflow:global_step/sec: 484.903
INFO:tensorflow:loss = 5.36791e+12, step = 101 (0.207 sec)
INFO:tensorflow:global_step/sec: 501.379
INFO:tensorflow:loss = 8.15759e+12, step = 201 (0.199 sec)
INFO:tensorflow:global_step/sec: 484.609
INFO:tensorflow:loss = 6.58913e+12, step = 301 (0.207 sec)
INFO:tensorflow:global_step/sec: 466.574
INFO:tensorflow:loss = 7.17105e+12, step = 401 (0.214 sec)
INFO:tensorflow:global_step/sec: 494.954
INFO:tensorflow:loss = 7.42304e+12, step = 501 (0.202 sec)
INFO:tensorflow:global_step/sec: 498.035
INFO:tensorflow:loss = 8.24344e+12, step = 601 (0.201 sec)
INFO:tensorflow:global_step/sec: 436.382
INFO:tensorflow:loss = 6.82244e+12, step = 701 (0.229 sec)
INFO:tensorflow:global_step/sec: 489.584
INFO:tensorflow:loss = 5.42115e+12, step = 801 (0.2

INFO:tensorflow:global_step/sec: 494.274
INFO:tensorflow:loss = 7.34666e+12, step = 8001 (0.203 sec)
INFO:tensorflow:global_step/sec: 403.685
INFO:tensorflow:loss = 7.4011e+12, step = 8101 (0.248 sec)
INFO:tensorflow:global_step/sec: 441.005
INFO:tensorflow:loss = 6.6196e+12, step = 8201 (0.226 sec)
INFO:tensorflow:global_step/sec: 450.3
INFO:tensorflow:loss = 7.50332e+12, step = 8301 (0.222 sec)
INFO:tensorflow:global_step/sec: 445.533
INFO:tensorflow:loss = 6.19834e+12, step = 8401 (0.224 sec)
INFO:tensorflow:global_step/sec: 486.824
INFO:tensorflow:loss = 6.94903e+12, step = 8501 (0.205 sec)
INFO:tensorflow:global_step/sec: 463.1
INFO:tensorflow:loss = 6.88402e+12, step = 8601 (0.216 sec)
INFO:tensorflow:global_step/sec: 420.389
INFO:tensorflow:loss = 6.62559e+12, step = 8701 (0.238 sec)
INFO:tensorflow:global_step/sec: 473.157
INFO:tensorflow:loss = 7.00063e+12, step = 8801 (0.211 sec)
INFO:tensorflow:global_step/sec: 500.989
INFO:tensorflow:loss = 7.25458e+12, step = 8901 (0.200 s

INFO:tensorflow:global_step/sec: 408.23
INFO:tensorflow:loss = 7.68663e+12, step = 16101 (0.245 sec)
INFO:tensorflow:global_step/sec: 433.071
INFO:tensorflow:loss = 6.6324e+12, step = 16201 (0.231 sec)
INFO:tensorflow:global_step/sec: 505.605
INFO:tensorflow:loss = 6.1776e+12, step = 16301 (0.198 sec)
INFO:tensorflow:global_step/sec: 368.833
INFO:tensorflow:loss = 7.02836e+12, step = 16401 (0.271 sec)
INFO:tensorflow:global_step/sec: 439.967
INFO:tensorflow:loss = 6.22151e+12, step = 16501 (0.227 sec)
INFO:tensorflow:global_step/sec: 435.635
INFO:tensorflow:loss = 6.39377e+12, step = 16601 (0.230 sec)
INFO:tensorflow:global_step/sec: 448.592
INFO:tensorflow:loss = 7.48419e+12, step = 16701 (0.223 sec)
INFO:tensorflow:global_step/sec: 471.511
INFO:tensorflow:loss = 8.14297e+12, step = 16801 (0.212 sec)
INFO:tensorflow:global_step/sec: 422.176
INFO:tensorflow:loss = 8.73787e+12, step = 16901 (0.237 sec)
INFO:tensorflow:global_step/sec: 473.444
INFO:tensorflow:loss = 7.50502e+12, step = 1

INFO:tensorflow:global_step/sec: 429.457
INFO:tensorflow:loss = 7.42331e+12, step = 24201 (0.233 sec)
INFO:tensorflow:global_step/sec: 455.04
INFO:tensorflow:loss = 6.43905e+12, step = 24301 (0.221 sec)
INFO:tensorflow:global_step/sec: 446.96
INFO:tensorflow:loss = 6.95028e+12, step = 24401 (0.223 sec)
INFO:tensorflow:global_step/sec: 439.945
INFO:tensorflow:loss = 6.19744e+12, step = 24501 (0.227 sec)
INFO:tensorflow:global_step/sec: 461.551
INFO:tensorflow:loss = 7.27618e+12, step = 24601 (0.217 sec)
INFO:tensorflow:global_step/sec: 456.233
INFO:tensorflow:loss = 7.49603e+12, step = 24701 (0.219 sec)
INFO:tensorflow:global_step/sec: 423.587
INFO:tensorflow:loss = 7.52358e+12, step = 24801 (0.236 sec)
INFO:tensorflow:global_step/sec: 424.744
INFO:tensorflow:loss = 6.46045e+12, step = 24901 (0.235 sec)
INFO:tensorflow:global_step/sec: 472.59
INFO:tensorflow:loss = 6.50065e+12, step = 25001 (0.212 sec)
INFO:tensorflow:global_step/sec: 457.878
INFO:tensorflow:loss = 6.46334e+12, step = 2

INFO:tensorflow:global_step/sec: 405.47
INFO:tensorflow:loss = 6.93309e+12, step = 32301 (0.247 sec)
INFO:tensorflow:global_step/sec: 434.962
INFO:tensorflow:loss = 6.26062e+12, step = 32401 (0.230 sec)
INFO:tensorflow:global_step/sec: 468.803
INFO:tensorflow:loss = 5.46808e+12, step = 32501 (0.213 sec)
INFO:tensorflow:global_step/sec: 323.653
INFO:tensorflow:loss = 6.0364e+12, step = 32601 (0.309 sec)
INFO:tensorflow:global_step/sec: 431.874
INFO:tensorflow:loss = 6.79373e+12, step = 32701 (0.232 sec)
INFO:tensorflow:global_step/sec: 467.336
INFO:tensorflow:loss = 6.48859e+12, step = 32801 (0.214 sec)
INFO:tensorflow:global_step/sec: 471.107
INFO:tensorflow:loss = 7.38409e+12, step = 32901 (0.212 sec)
INFO:tensorflow:global_step/sec: 456.29
INFO:tensorflow:loss = 5.71668e+12, step = 33001 (0.219 sec)
INFO:tensorflow:global_step/sec: 453.147
INFO:tensorflow:loss = 6.81753e+12, step = 33101 (0.221 sec)
INFO:tensorflow:global_step/sec: 463.986
INFO:tensorflow:loss = 7.20457e+12, step = 3

...

INFO:tensorflow:global_step/sec: 469.192
INFO:tensorflow:loss = 7.2273e+12, step = 64701 (0.213 sec)
INFO:tensorflow:global_step/sec: 481.955
INFO:tensorflow:loss = 7.55999e+12, step = 64801 (0.208 sec)
INFO:tensorflow:global_step/sec: 494.822
INFO:tensorflow:loss = 7.93799e+12, step = 64901 (0.202 sec)
INFO:tensorflow:global_step/sec: 508.016
INFO:tensorflow:loss = 7.1655e+12, step = 65001 (0.197 sec)
INFO:tensorflow:global_step/sec: 502.745
INFO:tensorflow:loss = 7.18733e+12, step = 65101 (0.199 sec)
INFO:tensorflow:global_step/sec: 496.719
INFO:tensorflow:loss = 7.83841e+12, step = 65201 (0.201 sec)
INFO:tensorflow:global_step/sec: 488.15
INFO:tensorflow:loss = 6.68177e+12, step = 65301 (0.205 sec)
INFO:tensorflow:global_step/sec: 474.417
INFO:tensorflow:loss = 7.09604e+12, step = 65401 (0.211 sec)
INFO:tensorflow:global_step/sec: 579.337
INFO:tensorflow:loss = 6.89547e+12, step = 65501 (0.174 sec)
INFO:tensorflow:global_step/sec: 492.779
INFO:tensorflow:loss = 7.0758e+12, step = 65

INFO:tensorflow:global_step/sec: 460.691
INFO:tensorflow:loss = 7.04397e+12, step = 72801 (0.217 sec)
INFO:tensorflow:global_step/sec: 472.219
INFO:tensorflow:loss = 6.30671e+12, step = 72901 (0.212 sec)
INFO:tensorflow:global_step/sec: 432.758
INFO:tensorflow:loss = 6.18465e+12, step = 73001 (0.231 sec)
INFO:tensorflow:global_step/sec: 465.162
INFO:tensorflow:loss = 7.67719e+12, step = 73101 (0.215 sec)
INFO:tensorflow:global_step/sec: 437.434
INFO:tensorflow:loss = 6.52678e+12, step = 73201 (0.229 sec)
INFO:tensorflow:global_step/sec: 440.622
INFO:tensorflow:loss = 8.63904e+12, step = 73301 (0.227 sec)
INFO:tensorflow:global_step/sec: 440.205
INFO:tensorflow:loss = 7.83975e+12, step = 73401 (0.227 sec)
INFO:tensorflow:global_step/sec: 491.342
INFO:tensorflow:loss = 6.44406e+12, step = 73501 (0.204 sec)
INFO:tensorflow:global_step/sec: 450.765
INFO:tensorflow:loss = 5.58994e+12, step = 73601 (0.222 sec)
INFO:tensorflow:global_step/sec: 484.776
INFO:tensorflow:loss = 6.92513e+12, step 

INFO:tensorflow:global_step/sec: 449.454
INFO:tensorflow:loss = 6.86523e+12, step = 80901 (0.222 sec)
INFO:tensorflow:global_step/sec: 429.893
INFO:tensorflow:loss = 6.8047e+12, step = 81001 (0.233 sec)
INFO:tensorflow:global_step/sec: 453.163
INFO:tensorflow:loss = 7.39297e+12, step = 81101 (0.221 sec)
INFO:tensorflow:global_step/sec: 468.39
INFO:tensorflow:loss = 6.46427e+12, step = 81201 (0.214 sec)
INFO:tensorflow:global_step/sec: 400.771
INFO:tensorflow:loss = 6.39602e+12, step = 81301 (0.250 sec)
INFO:tensorflow:global_step/sec: 471.565
INFO:tensorflow:loss = 6.05394e+12, step = 81401 (0.212 sec)
INFO:tensorflow:global_step/sec: 459.152
INFO:tensorflow:loss = 6.73262e+12, step = 81501 (0.217 sec)
INFO:tensorflow:global_step/sec: 455.741
INFO:tensorflow:loss = 7.1035e+12, step = 81601 (0.219 sec)
INFO:tensorflow:global_step/sec: 439.553
INFO:tensorflow:loss = 7.48134e+12, step = 81701 (0.228 sec)
INFO:tensorflow:global_step/sec: 462.839
INFO:tensorflow:loss = 7.26604e+12, step = 8

INFO:tensorflow:global_step/sec: 412.677
INFO:tensorflow:loss = 6.76332e+12, step = 89001 (0.242 sec)
INFO:tensorflow:global_step/sec: 453.772
INFO:tensorflow:loss = 8.4549e+12, step = 89101 (0.220 sec)
INFO:tensorflow:global_step/sec: 434.069
INFO:tensorflow:loss = 6.88253e+12, step = 89201 (0.230 sec)
INFO:tensorflow:global_step/sec: 441.386
INFO:tensorflow:loss = 7.48034e+12, step = 89301 (0.227 sec)
INFO:tensorflow:global_step/sec: 464.611
INFO:tensorflow:loss = 6.25419e+12, step = 89401 (0.215 sec)
INFO:tensorflow:global_step/sec: 458.001
INFO:tensorflow:loss = 6.66399e+12, step = 89501 (0.218 sec)
INFO:tensorflow:global_step/sec: 434.7
INFO:tensorflow:loss = 7.15962e+12, step = 89601 (0.230 sec)
INFO:tensorflow:global_step/sec: 443.898
INFO:tensorflow:loss = 7.69908e+12, step = 89701 (0.225 sec)
INFO:tensorflow:global_step/sec: 459.126
INFO:tensorflow:loss = 7.15202e+12, step = 89801 (0.218 sec)
INFO:tensorflow:global_step/sec: 459.521
INFO:tensorflow:loss = 6.89459e+12, step = 8

INFO:tensorflow:global_step/sec: 516.323
INFO:tensorflow:loss = 7.74299e+12, step = 97101 (0.194 sec)
INFO:tensorflow:global_step/sec: 484.562
INFO:tensorflow:loss = 6.71345e+12, step = 97201 (0.206 sec)
INFO:tensorflow:global_step/sec: 456.512
INFO:tensorflow:loss = 6.67802e+12, step = 97301 (0.219 sec)
INFO:tensorflow:global_step/sec: 483.973
INFO:tensorflow:loss = 8.09441e+12, step = 97401 (0.206 sec)
INFO:tensorflow:global_step/sec: 458.035
INFO:tensorflow:loss = 6.91491e+12, step = 97501 (0.218 sec)
INFO:tensorflow:global_step/sec: 481.01
INFO:tensorflow:loss = 6.9174e+12, step = 97601 (0.208 sec)
INFO:tensorflow:global_step/sec: 441.224
INFO:tensorflow:loss = 7.65542e+12, step = 97701 (0.227 sec)
INFO:tensorflow:global_step/sec: 466.094
INFO:tensorflow:loss = 6.95759e+12, step = 97801 (0.215 sec)
INFO:tensorflow:global_step/sec: 484.522
INFO:tensorflow:loss = 6.23154e+12, step = 97901 (0.208 sec)
INFO:tensorflow:global_step/sec: 429.568
INFO:tensorflow:loss = 5.9051e+12, step = 9

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [241]:
pred_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, shuffle=False)

In [242]:
pred_gen = model.predict(input_fn=pred_input_fn)

In [243]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /var/folders/xt/qvsnsqc941q950ghsjhyjh3r0000gn/T/tmpq9qdberp/model.ckpt-100000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [244]:
final_preds = [pred['predictions'] for pred in predictions]

In [245]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, final_preds) ** 0.5

237791.71643398772

97921.93181985477

# Great Job!